In [1]:
import pickle
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
tqdm.pandas()
%matplotlib inline

In [2]:
imdb_filenames = ['name.basics.tsv.gz', 'title.akas.tsv.gz', 
                  'title.basics.tsv.gz', 'title.crew.tsv.gz', 
                  'title.principals.tsv.gz', 'title.ratings.tsv.gz']

In [3]:
title_basics_df = pd.read_csv('data/title.basics.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
title_cast_df = pd.read_csv('data/title.principals.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"',
                           usecols = ['tconst', 'nconst'])
title_ratings_df = pd.read_csv('data/title.ratings.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')

In [4]:
title_basics_df = title_basics_df[title_basics_df['startYear'] !='\\N']

In [5]:
title_basics_df = title_basics_df[title_basics_df['runtimeMinutes'] !='\\N']

In [6]:
title_basics_df.dtypes

tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult           object
startYear         object
endYear           object
runtimeMinutes    object
genres            object
dtype: object

In [7]:
title_basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [8]:
title_basics_df['titleType'] = title_basics_df['titleType'].astype(str)
title_basics_df['isAdult'] = title_basics_df['isAdult'].astype(int)
title_basics_df['runtimeMinutes'] = title_basics_df['runtimeMinutes'].astype(int)
title_basics_df['startYear'] = title_basics_df['startYear'].astype(int)

In [9]:
title_basics_df['startYear'].unique()

array([1894, 1892, 1893, 1895, 1896, 1897, 1898, 1899, 1900, 1901, 1904,
       1902, 1903, 1905, 1912, 1907, 1908, 1910, 1906, 1909, 1914, 1913,
       1911, 1919, 1916, 1915, 1936, 1917, 1925, 1918, 1922, 1920, 1921,
       1923, 2019, 1924, 1927, 1929, 2000, 1926, 1993, 1935, 2014, 1928,
       2004, 1942, 1930, 2011, 1934, 1931, 1932, 1939, 1937, 1933, 1950,
       1938, 1951, 1945, 1946, 1940, 1944, 1949, 1947, 1941, 1952, 1970,
       1957, 1943, 1959, 1948, 2001, 1953, 2008, 1954, 1965, 1983, 1980,
       1973, 1961, 1995, 1955, 1962, 1958, 1956, 1977, 1960, 1964, 1967,
       1968, 1963, 1969, 1985, 1971, 1966, 2021, 1976, 1990, 1986, 1972,
       2020, 1979, 1974, 1978, 1981, 1988, 1975, 1989, 2009, 1987, 2010,
       2018, 1984, 1982, 1992, 1991, 1997, 1994, 1999, 2005, 1998, 2002,
       1996, 2017, 2016, 2006, 2007, 2003, 2012, 2013, 2015, 1891, 1888,
       1890, 2023, 2022, 2026, 1878, 2027, 2025, 1887, 1874, 2024, 1883,
       1881, 1885])

In [10]:
# 2015 - 2021
# 2005 - 2010
# 2011 - 2014

In [11]:
mask = ((title_basics_df['startYear'] >= 2011) &
        (title_basics_df['startYear'] <= 2021) &
        (title_basics_df['titleType'] == 'movie') &
        (title_basics_df['isAdult'] == 0) & 
        (title_basics_df['runtimeMinutes'] >80) &
        (title_basics_df['genres'] != '') &
        (title_basics_df['genres'] != 'Documentary'))

In [12]:
def clean_year(y):
    # Return year as an integer or 'NaN' if empty
    import numpy as np
    try:
        return int(y)
    except:
        return np.nan

def clean_genre(y):
    # Return only the first genre listed
    y = str(y)
    if y == '\\N':
        return ''
    return y.split(',')[0].strip()

title_basics_df.drop('endYear', axis=1, inplace=True)
title_basics_df['startYear'] = title_basics_df['startYear'].apply(clean_year)
title_basics_df['runtimeMinutes'] = title_basics_df['runtimeMinutes'].apply(clean_year)
title_basics_df['genres'] = title_basics_df['genres'].apply(clean_genre)
title_basics_df.dropna(inplace=True, how='any', subset=['startYear', 'runtimeMinutes'])

In [13]:
title_basics_df.shape

(2162551, 8)

In [14]:
title_basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,1,Documentary
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,5,Animation
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,4,Animation
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,12,Animation
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,1,Comedy


In [15]:
title_cast_df.head()

,tconst,nconst
0,tt0000001,nm1588970
1,tt0000001,nm0005690
2,tt0000001,nm0374658
3,tt0000002,nm0721526
4,tt0000002,nm1335271


In [16]:
title_cast_df.shape

(44252270, 2)

In [17]:
def f(df):
    keys, values = df.sort_values('tconst').values.T
    ukeys, index = np.unique(keys, True)
    arrays = np.split(values, index[1:])
    df2 = pd.DataFrame({'tconst':ukeys, 'nconst':[list(a) for a in arrays]})
    return df2

In [18]:
title_cast_df = f(title_cast_df)

In [19]:
title_cast_df.head()

,tconst,nconst
0,tt0000001,"[nm1588970, nm0005690, nm0374658]"
1,tt0000002,"[nm0721526, nm1335271]"
2,tt0000003,"[nm1335271, nm5442200, nm0721526, nm1770680]"
3,tt0000004,"[nm0721526, nm1335271]"
4,tt0000005,"[nm0443482, nm0653042, nm0005690, nm0249379]"


In [20]:
title_basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,1,Documentary
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,5,Animation
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,4,Animation
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,12,Animation
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,1,Comedy


In [21]:
titles = title_basics_df[mask].merge(title_cast_df, on='tconst')
titles = titles.merge(title_ratings_df, on='tconst')

In [22]:
titles.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,nconst,averageRating,numVotes
0,tt0019996,movie,Hongxia,Hongxia,0,2011,94,Action,"[nm0266662, nm0920761, nm0955817, nm0946300]",6.3,52
1,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama,"[nm0004372, nm0006166, nm1475059, nm0550881, n...",6.8,6464
2,tt0116991,movie,Mariette in Ecstasy,Mariette in Ecstasy,0,2019,101,Drama,"[nm0765477, nm0007037, nm0458356, nm0696895, n...",7.7,45
3,tt0137204,movie,Joe Finds Grace,Joe Finds Grace,0,2017,83,Adventure,"[nm0365480, nm0003210, nm0367762, nm0186322, n...",8.6,277
4,tt0137818,movie,Housesitter: The Night They Saved Siegfried's ...,Housesitter: The Night They Saved Siegfried's ...,0,2018,95,,"[nm10293681, nm0219703, nm0947053, nm0638396, ...",4.1,21


In [23]:
titles.shape

(53701, 11)

In [24]:
# with open('spider/data.pkl', 'wb') as picklefile:
#     pickle.dump(titles['tconst'].values, picklefile)

In [25]:
# titles = titles.merge(title_ratings_df, on='tconst')

In [26]:
titles.columns

Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'runtimeMinutes', 'genres', 'nconst', 'averageRating',
       'numVotes'],
      dtype='object')

In [27]:
titles['leadActor'] = titles['nconst'].apply(lambda x: x[0])

In [28]:
# Load scrapy json to my_data
with open('data/scraped/results_final1.json', 'r') as f:
    my_data = json.load(f)
    
with open('data/scraped/results_final2.json', 'r') as f:
    my_data2 = json.load(f)   

imdb_info1 = pd.DataFrame(my_data)
imdb_info2 = pd.DataFrame(my_data2)
imdb_info = imdb_info1.append(imdb_info2, ignore_index=True)

imdb_info.head()

,title_id,title,release,director,budget,opening,gross,worldwide_gross,metacritic_score,mpaa_rating
0,tt0293429,Mortal Kombat,23 April 2021 (USA),Simon McQuoid,,,,10700000,,R
1,tt0315642,Wazir,8 January 2016 (USA),Bejoy Nambiar,,586028,1124045,5633588,,Not Rated
2,tt0331314,Bunyan and Babe,12 January 2017 (USA),Louis Ross,,,,72060,,PG
3,tt0369610,Jurassic World,12 June 2015 (USA),Colin Trevorrow,150000000,208806270,652385625,1670516444,59,PG-13
4,tt0385887,Motherless Brooklyn,1 November 2019 (USA),Edward Norton,26000000,3500454,9277736,18477736,60,R


In [ ]:
imdb_info.shape

In [ ]:
# dict(imdb_info['mpaa_rating'].value_counts())

In [ ]:
rating_list = ['R', 'PG-13', 'PG', 'G', 'R-13', 'R-16']
imdb_mask = ((imdb_info['budget'] != '') &
             (imdb_info['opening'] != '') &
             (imdb_info['mpaa_rating'].isin(rating_list)))

In [ ]:
imdb_info = imdb_info[imdb_mask]

In [ ]:
imdb_info.shape

In [ ]:
imdb_info['budget'] = imdb_info['budget'].apply(int)
imdb_info['budget_mil'] = imdb_info['budget']/1000000.
imdb_info['opening'] = imdb_info['opening'].apply(int)
imdb_info['opening_mil'] = imdb_info['opening']/1000000.
imdb_info['release'] = pd.to_datetime(imdb_info['release'].apply(lambda x: x.split('(')[0].strip()))
imdb_info['tconst'] = imdb_info['title_id']
imdb_info.drop('title_id', inplace=True, axis=1)
imdb_info.head()

In [ ]:
imdb_info.to_csv('data/imdb_info.csv', index = False)

In [ ]:
imdb_info[imdb_info['title'].str.lower().str.contains('frozen')]

In [ ]:
# Merge IMDB Info with titles data
titles_all = imdb_info.merge(titles, on='tconst')

In [ ]:
titles_all.head()

In [ ]:
titles_all.shape

In [ ]:
col_names = ['tconst', 'nconst']
expanded_data = []
for idx, row in titles_all[col_names].iterrows():
    for name in row['nconst']:
        expanded_data.append([row['tconst'], name.strip()])
expanded_data = pd.DataFrame(expanded_data, columns=['tconst', 'nconst'])
expanded_data.head()

In [ ]:
name_basics_df = pd.read_csv('data/name.basics.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')

In [ ]:
name_basics_df.head()

In [ ]:
df_names = expanded_data.merge(name_basics_df, how='left', on='nconst')

In [ ]:
df_names.dropna(subset = ['primaryProfession'], inplace = True)

In [ ]:
df_names.head()

In [ ]:
# Clean Up (limit to single primary profession)
df_names['primaryProfession'] = df_names['primaryProfession'].apply(lambda x: x.split(',')[0].strip())
df_names.head()

In [ ]:
df_names.shape

In [ ]:
titles_all.drop('nconst', axis = 1, inplace = True)

In [ ]:
# merge this with titles_all dataframe
df_total = df_names.merge(titles_all, how='left', on='tconst')
df_total.head()

In [ ]:
df_total.columns

In [ ]:
df_total.shape

In [ ]:
# df_total[df_total['title'].str.contains('Frozen')]

In [ ]:
df_total.columns

In [ ]:
# df_total.to_csv('data/df_processed_total1.csv', index = False)